# Jupyter notebook to query the harvested metadata records from the IISG archive

This notebook makes it possible to get overviews and query the metadata records of the International Institute of Social History (IISG) Archives. It uses as source the file "converted.csv" obtained via metadata harvesting using the scripts in this repository (https://github.com/lilimelgar/iisg-metadata-overviews).  It contains MARC records from the OAIPMH endpoint. 
The file contains one record per row, and each marc property (field and subfield) is in a column.

Note: the data includes only metadata records at the "record" level, not the archival finding aids.

Created by Liliana Melgar (April, 2024).

# A. Set up

## A1. Import the required python libraries 
*(nothing to change)*

In [1]:
import pandas as pd
import numpy as np
import csv
import re

from IPython.display import display, HTML
from IPython.display import clear_output
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

# to add timestamp to file names
import time
# import os.path to add paths to files
import os

## A2. Set the path to the csv file 
*nothing to change if you cloned the repository. If you downloaded the file only ("archives_as_csv.csv"), then set here the path to where you have downloaded the file*

In [2]:
# path to where the transformed csv is located
data_directory = os.path.abspath(os.path.join('..', 'data'))
data_converted = os.path.join(data_directory, 'converted') #path to the repository folder where the csv file is located, if you have not cloned the repository, change the path here
data_downloads = os.path.join(data_directory, 'downloads') #path to the folder where the reports will be downloaded

## A3. Read the csv file as a pandas dataframe
*nothing to change here*

In [3]:
# read csv as dataframe
archive_df_v0 = pd.read_csv(f'{data_converted}/archives_as_csv.gzip', sep="\t", compression='gzip', low_memory=False)
# low_memory=False was set after this warning message: "/var/folders/3y/xbjxw0b94jxg6x2bcbyjsmmcgvnf7q/T/ipykernel_987/2912965462.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False."

# B. First overview and data preparation

## B1. First overview: all fields and data types
Execute the cell and view the general information of the data, which includes the Columns (marc properties with subfields), the Non-Null Count (i.e., how many cells have values; for example: if a cell says "1 non-null" it means that only one row has a value); and the Data type (object (i.e., a string or a combination of data types), a float or an integer).
- Keep in mind that the MARC labels have 3 characters, and that the fourth character can be an indicator or a subfield. For example: 1000 is Marc label 100 with indicator 0. And 100a is Marc label 100 with subfield a.

In [4]:
archive_df_v0.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5423 entries, 0 to 5422
Data columns (total 69 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   001     5423 non-null   object 
 1   003     5423 non-null   object 
 2   008     5423 non-null   object 
 3   040a    5423 non-null   object 
 4   040b    5423 non-null   object 
 5   040c    5423 non-null   object 
 6   040e    5415 non-null   object 
 7   041a    5423 non-null   object 
 8   1000    4 non-null      float64
 9   100a    2377 non-null   object 
 10  100c    1 non-null      object 
 11  100d    2 non-null      object 
 12  100e    2377 non-null   object 
 13  100q    1 non-null      object 
 14  1100    1 non-null      float64
 15  110a    2255 non-null   object 
 16  110b    478 non-null    object 
 17  110e    2255 non-null   object 
 18  245a    5423 non-null   object 
 19  245f    5416 non-null   object 
 20  245g    1367 non-null   object 
 21  300a    5423 non-null   object 
 22  

## B2. Optional (documentation)
Ideally, each field above would have a definition explaining what it means and what kind of values does it contain (in relation to the conventions for creating IISG metadata). That documentation can exist somewhere else (e.g., on Confluence), but this could be a place to start updating or writing those definitions since here one can see the data that they contain in detail.

- 001 : record Id
- 003
- 008
- 040a
- 040b
- 040c
- 040e
- 041a
- 1000
- 100a
- 100c
- 100d
- 100e
- 100q
- 1100
- 110a
- 110b
- 110e
- 245a
- 245f
- 245g
- 300a
- 300f
- 351a
- 500a
- 506a
- 520a
- 524a
- 530a
- 535a
- 540a
- 541a
- 544d
- 545a
- 546a
- 555a
- 561a
- 583a
- 6000
- 6002
- 6004
- 600a
- 600q
- 6102
- 6104
- 610a
- 610b
- 610e
- 6502
- 650a
- 6512
- 651a
- 6552
- 655a
- 7000
- 7004
- 700a
- 700e
- 7104
- 710a
- 710b
- 710e
- 852a
- 852b
- 852j
- 856q
- 856u
- 902a
- leader

## B3. Prepare the data for search
Because we know that the data doesn't have proper numerical values to be computed, we rather convert all values to strings in order to facilitate querying. This also includes filling in empty values with a standard string: "null"
*(nothing to change here)*

In [ ]:
# convert datatypes and fill in empty values
df_columns = archive_df_v0.columns
for column in df_columns:
    dataType = archive_df_v0.dtypes[column]
    if dataType == np.float64:
        archive_df_v0[column] = archive_df_v0[column].fillna('null')
        archive_df_v0[column] = archive_df_v0[column].astype(str)
    if dataType == object:
        archive_df_v0[column] = archive_df_v0[column].fillna('null')
        archive_df_v0[column] = archive_df_v0[column].astype(str)

In [ ]:
# create a copy
archive_df = archive_df_v0.copy()

In [ ]:
# Check again the general information of the data after having filled in the emtpy values and converted the data types
archive_df.info(verbose = True, show_counts = True)

# C. Get a glimpse of the data

## C1. First rows
Here you can see a sample of the records, one per line. You can change the value "10" to any other desired size for your sample, preferably not too big. You can also use "tail" instead of "head" to see the records in the last rows.
- Keep in mind to scroll horizontally and vertically to see the entire record.
- NaN means that the cell is empty.
- Arbitrarily, some cells above, we decided that the omega "Ω" would be the separator for multi-value cells. For example, this archive: ARCH00002 contains two persons as topics in field 600a, which show here separated by "Ω"

In [ ]:
archive_df.head(10)

## C2. Size (shape) of the data
Here you can see how many rows (first value) and how many columns (second value) are in the data.

In [ ]:
archive_df.shape

## C3. Unique values
Here you can see a general description of the data, including how many unique values are per column.

In [ ]:
# describe the dataframe
archive_df.describe()

# D. Check the values in one column (marc property)
At this point you may be curious to know which values are in one column. For example, 100e has only 3 unique values, which are those?
- You can change the field inside the quotation marcs for any other field of interest.

In [ ]:
archive_df['100e'].unique().tolist()

## D1. Create a subset with certain column(s)/field(s)
At this point you may have thought that you could perhaps correct some of the records which contain an inconsistent value. For example, in the first version of this data, if you queried above for "archive_df['100e'].unique()" you may have obtained these values: ['creator.', 'null', 'creator']. You may decide that you want to change "creator." into "creator". But for this, you need the TCN (record Id) numbers. The command below facilitates creating a subset with the TCN and the field of interest.


In [ ]:
# create subset with record Id and record of interest, here enter the name of the field(s) that you are interested in separated by commas, each field has to be within single quotation marks, e.g., archive_df[['001','100e', '110e']]
field_subset_df = archive_df[['001','100e']]
field_subset_df

In [ ]:
# check again the number of unique values in your subset
field_subset_df.describe()

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
name_file = 'archives_relator_field_100e'

field_subset_df.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
# field_subset_df.to_csv()

## D2. Create a subset of records with a certain value in a given column
You may also want to create a list of the records with a certain value in a given column, for example, for field 100e you got these unique values: ['creator.', 'null', 'creator']. You may want to get only the list of records that have "creator."

In [ ]:
# check if a string value exists in a column (the string is exactly the same)
query_value_exact = field_subset_df[field_subset_df['100e'] == 'creator.']
query_value_exact

In [ ]:
# check if a string value exists in a column (the string is approximately the same)
# you may want to find the records that have either "creator." (with dot) or "creator" without dot, but not the null values
# here it's possible to use regular expressions

query_value_aprox = field_subset_df[field_subset_df['100e'].str.contains("creator", case=False, regex=True)]
query_value_aprox

In [ ]:
# get some idea of how many rows are in this set
query_value_aprox.info(verbose = True, show_counts = True)

In [ ]:
# check again the number of unique values in your subset
query_value_aprox.describe()

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
name_file = 'archives_relator_field_100e_creator'

query_value_aprox.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
# query_value_aprox.to_csv()

# E. Create subsets using inverse query
You may need to create a report with all the records that do not contain a certain value. For example, because we used "null" to fill in all empty values, one could create a list with all the records that have a value in a certain column.

In [ ]:
# create a slice with the records that have non-null values in the column of interest
# Note: if you want to query the subset instead of the whole data, then replace "archive_df" with "field_subset_df" and run the cell again

query_inverse = archive_df[~archive_df['100e'].str.contains("null", case=False, regex=True)]

query_inverse.head(10)

In [ ]:
# get some info about the subset you got as a result of the query:
query_inverse.info(verbose=True, show_counts = True)

In [ ]:
# You may want to dowload the table above to an excel file for further inspection:

# choose any name for your file, the file will go to the ../data/downloads folder.
name_file = 'archives_relator_field_100e_notEmpty'

query_inverse.to_excel(f'{data_downloads}/{name_file}.xlsx')

## or download to csv
# query_inverse.to_csv()

# F. Query for a specific record
You may want to see the details of a specific record, this can be done in two ways:

In [ ]:
# 1. by using the index position. Example: This archive: 10622/ARCH00001 has index position 0. 
# This position can be seen in the left corner of the entire table (cell above in Section5: archive_df.head(10))
# We will query it using the entire version of the data, not the subset

# show record vertically using index position
query_recordIndex = archive_df.iloc[0]
query_recordIndex

In [ ]:
# 2. By using the record Id using the Marc field 001
query_recordId = archive_df[archive_df['001'] == '10622/ARCH00001']
query_recordId